### Running trained model on actual data 

In [46]:
import joblib
import pandas as pd
import numpy as np

In [47]:
model = joblib.load("model.pkl")
scaler = joblib.load("scaler.pkl")
le = joblib.load("label_encoder.pkl")

In [48]:
torque_df  = pd.read_csv(r'D:\m_bi_data1\simulated_data\torque_sensor.csv',  parse_dates=["timestamp"])
current_df = pd.read_csv(r'D:\m_bi_data1\simulated_data\current_draw_sensor.csv', parse_dates=["timestamp"])
temp_df    = pd.read_csv(r'D:\m_bi_data1\simulated_data\motor_temperature_sensor.csv', parse_dates=["timestamp"])
rot_df     = pd.read_csv(r'D:\m_bi_data1\simulated_data\rotations_sensor.csv', parse_dates=["timestamp"])
vib_df     = pd.read_csv(r'D:\m_bi_data1\simulated_data\vibration_sensor.csv', parse_dates=["timestamp"])

In [49]:
df = torque_df[["miner_id","timestamp","torque"]].merge(
    current_df[["miner_id","timestamp","current_draw"]], on=["miner_id","timestamp"]
).merge(
    temp_df[["miner_id","timestamp","motor_temperature"]], on=["miner_id","timestamp"]
).merge(
    rot_df[["miner_id","timestamp","rotations"]], on=["miner_id","timestamp"]
).merge(
    vib_df[["miner_id","timestamp","vibration_level"]], on=["miner_id","timestamp"]
)

In [50]:
df = df.rename(columns={
    "current_draw": "current",
    "motor_temperature": "temp",
    "vibration_level": "vibration"
})

In [51]:
df.head()

,miner_id,timestamp,torque,current,temp,rotations,vibration
0,1,2025-01-01 00:00:00,65.331051,10.679084,34.642580,1126.023118,1.378389
1,1,2025-01-01 00:10:00,51.970095,11.075572,59.982920,875.471690,0.542098
2,1,2025-01-01 00:20:00,77.475999,11.992081,31.986602,963.834389,1.368263
3,1,2025-01-01 00:30:00,85.533649,9.513321,53.655930,927.900655,0.000000
4,1,2025-01-01 00:40:00,61.099477,9.781333,51.922212,1097.196128,1.949811


In [52]:
model = joblib.load("model.pkl")
scaler = joblib.load("scaler.pkl")
le = joblib.load("label_encoder.pkl")

In [53]:
WINDOW_SIZE = 1000                 
STEP = 500                          
SENSORS = ["torque", "current", "temp", "rotations", "vibration"]

In [ ]:
features_list = []

for miner_id, miner_group in df.groupby("miner_id"):
    miner_group = miner_group.sort_values("timestamp").reset_index(drop=True)
    num_points = len(miner_group)

    for start in range(0, num_points - WINDOW_SIZE + 1, STEP):
        window = miner_group.iloc[start:start + WINDOW_SIZE]
        feat = {"miner_id": miner_id, "start_time": window["timestamp"].iloc[0], "end_time": window["timestamp"].iloc[-1]}

        for sensor in SENSORS:
            values = window[sensor].values
            feat[f"{sensor}_mean"] = np.mean(values)
            feat[f"{sensor}_std"] = np.std(values)
            feat[f"{sensor}_max"] = np.max(values)
            feat[f"{sensor}_min"] = np.min(values)
            feat[f"{sensor}_trend"] = np.polyfit(np.arange(len(values)), values, 1)[0]
            feat[f"{sensor}_energy"] = np.sum(values**2)

        features_list.append(feat)

features_df = pd.DataFrame(features_list)

features_df.head()

Feature table shape: (1130, 33)
   miner_id          start_time            end_time  torque_mean  torque_std  \
0         1 2025-01-01 00:00:00 2025-01-07 22:30:00    64.214076   23.949789   
1         1 2025-01-04 11:20:00 2025-01-11 09:50:00    64.539354   24.448410   
2         1 2025-01-07 22:40:00 2025-01-14 21:10:00    64.100695   25.422065   
3         1 2025-01-11 10:00:00 2025-01-18 08:30:00    63.733821   24.620394   
4         1 2025-01-14 21:20:00 2025-01-21 19:50:00    64.256661   24.647010   

   torque_max  torque_min  torque_trend  torque_energy  current_mean  ...  \
0  126.774088         0.0     -0.007409   4.697040e+06      9.897958  ...   
1  123.427043         0.0      0.007505   4.763053e+06      9.849528  ...   
2  123.427043         0.0     -0.008136   4.755181e+06      9.878304  ...   
3  121.153689         0.0      0.007722   4.668164e+06      9.907151  ...   
4  134.382320         0.0     -0.004655   4.736394e+06      9.965724  ...   

   rotations_max  rotati

In [55]:
X = features_df.drop(["miner_id","start_time","end_time"], axis=1)
X_scaled = scaler.transform(X)

probs = model.predict_proba(X_scaled)
prob_df = pd.DataFrame(probs, columns=le.classes_)

In [56]:
prob_df["miner_id"] = features_df["miner_id"]
prob_df["start_time"] = features_df["start_time"]
prob_df["end_time"] = features_df["end_time"]

In [57]:
cols = ["miner_id","start_time","end_time"] + list(le.classes_)
prob_df = prob_df[cols]

prob_df.shape

(1130, 46)

In [58]:
prob_df.head()

,miner_id,start_time,end_time,D1,D10,D11,D12,D13,D14,D2,...,S17,S18,S2,S3,S4,S5,S6,S7,S8,S9
0,1,2025-01-01 00:00:00,2025-01-07 22:30:00,0.040,0.015,0.025,0.015,0.02,0.070,0.005,...,0.0,0.0,0.040,0.005,0.01,0.06,0.00,0.030,0.0,0.005
1,1,2025-01-04 11:20:00,2025-01-11 09:50:00,0.045,0.005,0.010,0.010,0.02,0.045,0.065,...,0.0,0.0,0.105,0.000,0.01,0.02,0.01,0.025,0.0,0.000
2,1,2025-01-07 22:40:00,2025-01-14 21:10:00,0.040,0.015,0.015,0.015,0.02,0.070,0.005,...,0.0,0.0,0.040,0.005,0.01,0.06,0.00,0.030,0.0,0.005
3,1,2025-01-11 10:00:00,2025-01-18 08:30:00,0.045,0.005,0.010,0.010,0.02,0.040,0.055,...,0.0,0.0,0.105,0.000,0.01,0.02,0.01,0.025,0.0,0.000
4,1,2025-01-14 21:20:00,2025-01-21 19:50:00,0.040,0.015,0.025,0.015,0.02,0.070,0.010,...,0.0,0.0,0.035,0.010,0.01,0.06,0.00,0.030,0.0,0.005


In [60]:
prob_df.to_csv("miner_probabilities.csv", index=False)